In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Draw, PandasTools, Descriptors
from rdkit.Chem.Draw import IPythonConsole
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Ridge, Lasso, LinearRegression, SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from math import log10
from IPython.display import HTML
import pandas as pd
from rdkit.Chem.AtomPairs import Pairs, Utils
import seaborn as sns
import matplotlib.pyplot as plt
# Suppress warnings of Python library
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("ERb_IC50.csv")
df

,Unnamed: 0,SMILES,Standard Value,pIC50
0,CHEMBL81,O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2...,470.0,6.327902
1,CHEMBL4278269,Cc1cc(-c2c[se]cc2-c2ccc(OCCN3CCCC3)c(C)c2)ccc1O,5.0,8.301030
2,CHEMBL188957,CCCc1cc(O)cc2nc(-c3ccc(O)cc3)oc12,11.0,7.958607
3,CHEMBL189706,CCOC(=O)c1cc(O)cc2nc(-c3ccc(O)cc3)oc12,190.0,6.721246
4,CHEMBL188230,O=Cc1cc(O)cc2nc(-c3ccc(O)cc3)oc12,59.0,7.229148
...,...,...,...,...
1551,CHEMBL3360846,O=C1[C@@H](c2ccc3ccccc3c2)[C@H](c2ccc(O)cc2)N1...,1100.0,5.958607
1552,CHEMBL4637118,COC(=O)C1=C(N)Oc2cc(O)ccc2C1c1ccccc1,1590.0,5.798603
1553,CHEMBL370458,N#CC1=C(N)Oc2cc(O)ccc2C1c1ccccc1,8390.0,5.076238
1554,CHEMBL4643970,COC(=O)C1=C(N)Oc2cc(O)ccc2C1c1ccc(OCCN2CCOCC2)cc1,630.0,6.200659


In [3]:
#　dfのSMILES列を参照してMolオブジェクト列をデータフレームに加える #https://insilico-notebook.com/smiles-to-desc-df/
PandasTools.AddMoleculeColumnToFrame(df,'SMILES')
#　Molオブジェクトが作成できたか確認
print(df.shape)
print(df.isnull().sum()) 
#　ROMolが作成できなかったものを確認
print(df[df.ROMol.isnull()])

#　欠損行の除去
df = df.dropna() 


(1556, 5)
Unnamed: 0        0
SMILES            0
Standard Value    0
pIC50             0
ROMol             0
dtype: int64
Empty DataFrame
Columns: [Unnamed: 0, SMILES, Standard Value, pIC50, ROMol]
Index: []


In [4]:
for i,j in Descriptors.descList:
    df[i] = df.ROMol.map(j)

df.shape
df['MW'] = df.ROMol.map(Descriptors.MolWt)
df['MW'].describe()

count    1556.000000
mean      366.029582
std        96.613238
min       142.110000
25%       285.255000
50%       340.949500
75%       449.572000
max       778.987000
Name: MW, dtype: float64

In [5]:
df = df.dropna(how='any', axis=0)
df.head

<bound method NDFrame.head of          Unnamed: 0                                             SMILES  \
0          CHEMBL81  O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2...   
2      CHEMBL188957                  CCCc1cc(O)cc2nc(-c3ccc(O)cc3)oc12   
3      CHEMBL189706             CCOC(=O)c1cc(O)cc2nc(-c3ccc(O)cc3)oc12   
4      CHEMBL188230                  O=Cc1cc(O)cc2nc(-c3ccc(O)cc3)oc12   
5      CHEMBL188528                   Oc1ccc(-c2nc3cc(O)cc(Br)c3o2)cc1   
...             ...                                                ...   
1551  CHEMBL3360846  O=C1[C@@H](c2ccc3ccccc3c2)[C@H](c2ccc(O)cc2)N1...   
1552  CHEMBL4637118               COC(=O)C1=C(N)Oc2cc(O)ccc2C1c1ccccc1   
1553   CHEMBL370458                   N#CC1=C(N)Oc2cc(O)ccc2C1c1ccccc1   
1554  CHEMBL4643970  COC(=O)C1=C(N)Oc2cc(O)ccc2C1c1ccc(OCCN2CCOCC2)cc1   
1555  CHEMBL4647883            COC(=O)C1=C(N)Oc2cc(O)ccc2C1c1ccc(O)cc1   

      Standard Value     pIC50  \
0              470.0  6.327902   
2            

In [6]:
#dfa = df.drop(df.columns[[0,1,2,3,4,214]], axis = 1)
dfa = df.drop(df.columns[[0,1,2,3,4]], axis = 1)
print(dfa)

      MaxEStateIndex  MinEStateIndex  MaxAbsEStateIndex  MinAbsEStateIndex  \
0          13.646561       -0.083443          13.646561           0.083443   
2           9.739204        0.200469           9.739204           0.200469   
3          11.947139       -0.586291          11.947139           0.096195   
4          10.931167       -0.041522          10.931167           0.041522   
5           9.482571        0.124936           9.482571           0.124936   
...              ...             ...                ...                ...   
1551       13.248370       -0.307452          13.248370           0.017635   
1552       12.148264       -0.536965          12.148264           0.014997   
1553        9.552529       -0.272639           9.552529           0.082827   
1554       12.468625       -0.562905          12.468625           0.040937   
1555       12.165818       -0.597870          12.165818           0.030115   

           qed    MolWt  HeavyAtomMolWt  ExactMolWt  NumValence

In [7]:
dfa = dfa.iloc[:, 0:376].apply(lambda x: (x-x.mean())/x.std(ddof=1), axis=0)
dfa.head

<bound method NDFrame.head of       MaxEStateIndex  MinEStateIndex  MaxAbsEStateIndex  MinAbsEStateIndex  \
0           1.198892        0.419192           1.198892          -0.284867   
2          -1.008182        0.672968          -1.008182           0.707316   
3           0.238972       -0.030282           0.238972          -0.176750   
4          -0.334901        0.456663          -0.334901          -0.640288   
5          -1.153141        0.605453          -1.153141           0.066926   
...              ...             ...                ...                ...   
1551        0.973973        0.218960           0.973973          -0.842810   
1552        0.352577        0.013808           0.352577          -0.865175   
1553       -1.113626        0.250077          -1.113626          -0.290091   
1554        0.533534       -0.009379           0.533534          -0.645245   
1555        0.362493       -0.040633           0.362493          -0.737001   

           qed     MolWt  HeavyAt

In [8]:
dfa = dfa.dropna(how='any', axis=1)
dfa.head

<bound method NDFrame.head of       MaxEStateIndex  MinEStateIndex  MaxAbsEStateIndex  MinAbsEStateIndex  \
0           1.198892        0.419192           1.198892          -0.284867   
2          -1.008182        0.672968          -1.008182           0.707316   
3           0.238972       -0.030282           0.238972          -0.176750   
4          -0.334901        0.456663          -0.334901          -0.640288   
5          -1.153141        0.605453          -1.153141           0.066926   
...              ...             ...                ...                ...   
1551        0.973973        0.218960           0.973973          -0.842810   
1552        0.352577        0.013808           0.352577          -0.865175   
1553       -1.113626        0.250077          -1.113626          -0.290091   
1554        0.533534       -0.009379           0.533534          -0.645245   
1555        0.362493       -0.040633           0.362493          -0.737001   

           qed     MolWt  HeavyAt

In [9]:
pIC50s = df.loc[:,['pIC50']]
pIC50s = pIC50s.dropna(how='any', axis=1)
X = dfa.values
y = pIC50s.values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [11]:
#autoscaled_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std(ddof=1)
autoscaled_X_train = X_train

In [17]:
dfytr = pd.DataFrame(autoscaled_y_train)
dfytr

,0
0,0.992026
1,-0.249621
2,-1.599027
3,-0.376321
4,1.315335
...,...
1147,0.687153
1148,1.386141
1149,1.672274
1150,1.963964


In [33]:
print(type(autoscaled_X_train))

<class 'numpy.ndarray'>


In [13]:
dfXtr = pd.DataFrame(autoscaled_X_train)
dfXtr

,0,1,2,3,4,5,6,7,8,9,...,179,180,181,182,183,184,185,186,187,188
0,-0.850528,0.109903,-0.850528,-0.585567,-0.077853,0.555396,0.577891,0.556948,0.639474,-0.665347,...,-0.367699,-0.159219,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,0.555396
1,-0.409376,0.416269,-0.409376,-0.257150,-2.152720,1.043202,1.136135,1.044143,0.853655,-0.137234,...,2.109430,-0.159219,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,1.043202
2,0.628017,0.258387,0.628017,-0.704350,0.845069,-0.875613,-0.848676,-0.874265,-0.806245,0.661066,...,-0.367699,-0.159219,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,-0.875613
3,1.414508,-3.298109,1.414508,-0.821286,-0.135453,1.088164,1.162031,1.084723,0.693020,1.732497,...,-0.367699,6.276575,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,1.088164
4,0.504248,0.009141,0.504248,-0.984882,-0.768959,-0.834804,-0.759885,-0.833344,-0.806245,0.355532,...,-0.367699,-0.159219,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,-0.834804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,-1.005685,0.738347,-1.005685,1.327441,-0.525720,0.235123,0.154895,0.236386,0.425294,-1.019576,...,-0.367699,-0.159219,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,0.235123
1148,-0.889187,0.431851,-0.889187,-0.478916,-0.024534,0.525236,0.466869,0.526605,0.693020,-1.019836,...,-0.367699,-0.159219,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,0.525236
1149,1.423748,0.092203,1.423748,-0.580773,1.312320,-0.161776,-0.103504,-0.169916,-0.699155,0.365599,...,-0.367699,-0.159219,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,-0.161776
1150,1.376363,0.210668,1.376363,0.920602,-0.408305,0.835970,0.812244,0.837513,0.960745,-0.897507,...,-0.367699,-0.159219,-0.108858,-0.088707,-0.072334,-0.136219,-0.292685,-0.153238,-0.080925,0.835970


In [12]:
# -*- coding: utf-8 -*- 
# %reset -f
"""
@author: Hiromasa Kaneko
"""
# Demonstration of (binary) Genetic Algorithm-based Support Vector Regression (GASVR)


import random
from deap import base
from deap import creator
from deap import tools
#from sklearn import datasets
from sklearn import model_selection
from sklearn import svm

# settings
number_of_population = 100
number_of_generation = 150
svr_c_2_range = (-5, 10)
svr_epsilon_2_range = (-10, 0)
svr_gamma_2_range = (-20, 10)
fold_number = 5
probability_of_crossover = 0.5
probability_of_mutation = 0.2
threshold_of_variable_selection = 0.5

# generate sample dataset
#X_train, y_train = datasets.make_regression(n_samples=100, n_features=300, n_informative=10, noise=10, random_state=0)





In [13]:
# autoscaling
#autoscaled_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
#autoscaled_y_train = (y_train - y_train.mean()) / y_train.std(ddof=1)

#autoscaled_y_train = y

# GASVR
creator.create('FitnessMax', base.Fitness, weights=(1.0,))  # for minimization, set weights as (-1.0,)
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
min_boundary = np.zeros(X_train.shape[1] + 3)
max_boundary = np.ones(X_train.shape[1] + 3) * 1.0
min_boundary[-3] = svr_c_2_range[0]
min_boundary[-2] = svr_epsilon_2_range[0]
min_boundary[-1] = svr_gamma_2_range[0]
max_boundary[-3] = svr_c_2_range[1]
max_boundary[-2] = svr_epsilon_2_range[1]
max_boundary[-1] = svr_gamma_2_range[1]

In [14]:
def create_ind_uniform(min_boundary, max_boundary):
    index = []
    for min, max in zip(min_boundary, max_boundary): # for ,, in zip ; loop against multiple sequence
        index.append(random.uniform(min, max))
    return index


toolbox.register('create_ind', create_ind_uniform, min_boundary, max_boundary)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.create_ind)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)


def evalOneMax(individual):
    individual_array = np.array(individual)
    selected_X_variable_numbers = np.where(individual_array[:-3] > threshold_of_variable_selection)[0]
    selected_autoscaled_X_train = autoscaled_X_train[:, selected_X_variable_numbers]
    if len(selected_X_variable_numbers):
        # cross-validation
        model_in_cv = svm.SVR(kernel='rbf', C=2 ** round(individual_array[-3]), #**: exponentiation
                              epsilon=2 ** round(individual_array[-2]), gamma=2 ** round(individual_array[-1]))
        estimated_y_train_in_cv = model_selection.cross_val_predict(model_in_cv, selected_autoscaled_X_train,
                                                                    autoscaled_y_train, cv=fold_number)
        estimated_y_train_in_cv = estimated_y_train_in_cv * y_train.std(ddof=1) + y_train.mean()
        value = 1 - sum((y_train - estimated_y_train_in_cv) ** 2) / sum((y_train - y_train.mean()) ** 2)
    else:
        value = -999

    return value,


toolbox.register('evaluate', evalOneMax) #ex, may change evalOneMax to evaluate
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05) #ex. tools.mutGaussian, mu=0, sigma=1, indpb=0.1
toolbox.register('select', tools.selTournament, tournsize=3)



In [15]:
# random.seed(100)
random.seed()
pop = toolbox.population(n=number_of_population)

print('Start of evolution')

fitnesses = list(map(toolbox.evaluate, pop)) #fitnesses = affect evaluate to n of pop, and convert to list
for ind, fit in zip(pop, fitnesses): #ind ; index
    ind.fitness.values = fit

print('  Evaluated %i individuals' % len(pop))
for generation in range(number_of_generation):
    print('-- Generation {0} --'.format(generation + 1))

    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]): #crossover offspring if random_value <crossover_value
        if random.random() < probability_of_crossover:
            toolbox.mate(child1, child2)
            del child1.fitness.values # del;delete
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < probability_of_mutation: #mutate offspring if random_value <mutetion_value
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    print('  Evaluated %i individuals' % len(invalid_ind))

    pop[:] = offspring
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x * x for x in fits)
    std = abs(sum2 / length - mean ** 2) ** 0.5

    print('  Min %s' % min(fits))
    print('  Max %s' % max(fits))
    print('  Avg %s' % mean)
    print('  Std %s' % std)

print('-- End of (successful) evolution --')

best_individual = tools.selBest(pop, 1)[0]
best_individual_array = np.array(best_individual)
selected_X_variable_numbers = np.where(best_individual_array[:-3] > threshold_of_variable_selection)[0]
print('Selected variables : %s, %s' % (selected_X_variable_numbers, best_individual.fitness.values))
print('C : 2 ** {0}'.format(round(best_individual_array[-3])))
print('Epsilon : 2 ** {0}'.format(round(best_individual_array[-2])))
print('Gamma : 2 ** {0}'.format(round(best_individual_array[-1])))


Start of evolution
  Evaluated 100 individuals
-- Generation 1 --


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()